$\newcommand{\xbf}{{\bf x}}
\newcommand{\ybf}{{\bf y}}
\newcommand{\wbf}{{\bf w}}
\newcommand{\Ibf}{\mathbf{I}}
\newcommand{\Xbf}{\mathbf{X}}
\newcommand{\Rbb}{\mathbb{R}}
\newcommand{\vec}[1]{\left[\begin{array}{c}#1\end{array}\right]}
$

# Introduction aux réseaux de neurones : TD #2  (partie 3)
Matériel de cours rédigé par Pascal Germain, 2018
************

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn

### Chargement des données «MNIST» et pré-traitements

In [ ]:
def charger_mnist(repertoire, etiquettes=None, max_par_etiquettes=None):
    if etiquettes is None:
         etiquettes = np.arange(10)
    images_list = [None] * len(etiquettes)
    labels_list = [None] * len(etiquettes)
    for i, val in enumerate(etiquettes):
        nom_fichier = repertoire_mnist + f'mnist_{val}.gz'
        images_list[i] = np.genfromtxt(nom_fichier, max_rows=max_par_etiquettes, dtype=np.float32)
        nb = images_list[i].shape[0]

        labels_list[i] = i*np.ones(nb, dtype=np.int64)
        print(val, ':', nb, 'images')
        
    x = np.vstack(images_list)
    y = np.concatenate(labels_list)
    print('Total :', len(y), 'images')
    return x, y

In [ ]:
repertoire_mnist = 'mnist/' # Modifier le répertoire au besoin
data_x, data_y = charger_mnist(repertoire_mnist, etiquettes=None, max_par_etiquettes=1000)
data_x = data_x / 255

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.5, random_state=42)
print('train_x:', train_x.shape)
print('test_x:', test_x.shape)
print('train_y:', train_y.shape)
print('test_y:', test_y.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_x)
train_x_prime = scaler.transform(train_x)
test_x_prime = scaler.transform(test_x)

### Créons un réseau de neurones

La classe suivante a pour but d'éviter de recopier le même code pour lors de nos différents essai. Ce réseau doit être initialisé avec un object `architecture`, possédant minimalement une méthode `propagation` et une méthode `parametres`.

In [ ]:
from itertools import chain
from torch.utils.data import TensorDataset, DataLoader

class ReseauClassifGenerique:
    def __init__(self, architecture, eta=0.4, alpha=0.1, nb_epoques=10, taille_batch=32, seed=None):
        self.architecture = architecture
        self.eta = eta
        self.alpha = alpha
        self.nb_epoques = nb_epoques
        self.taille_batch = taille_batch
        self.seed = seed
        
    def fit(self, x, y):
        if self.seed is not None:
            torch.manual_seed(self.seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.int64)
               
        nb_sorties = len(torch.unique(y))
        sampler = DataLoader(TensorDataset(x, y), batch_size=self.taille_batch, shuffle=True) 

        perte_logistique = nn.NLLLoss()
        
        optimizer = torch.optim.SGD(chain(*self.architecture.parametres()), 
                                    lr=self.eta, momentum=self.alpha)
        
        self.liste_objectif = list()
    
        for t in range(self.nb_epoques):
            liste_pertes = list()
            for batch_x, batch_y in sampler:
                 
                y_pred = self.architecture.propagation(batch_x, apprentissage=True)
                perte = perte_logistique(y_pred, batch_y)

                optimizer.zero_grad()
                perte.backward()
                optimizer.step()
                
                liste_pertes.append(perte.item())
                
            perte_moyenne = np.mean(liste_pertes)
            print(t, ':', perte_moyenne)
            self.liste_objectif.append(perte_moyenne)
                
                
    def predict(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        pred = self.architecture.propagation(x, apprentissage=False)
        pred = torch.argmax(pred, dim=1)
        return np.array(pred.detach())



Voici un exemple de classe à partir de laquelle nous créerons des objets `architecture`.

In [ ]:
class UneArchiPourMNIST:
    def __init__(self, nb_filtres=32, taille_noyau=3):
        # Créons une couche de convolution 
        self.modele_conv = nn.Sequential(
            nn.Conv2d(1, nb_filtres, kernel_size=taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # La convolution est suivie d'une couche de sortie 
        self.nb_neurones_du_milieu = nb_filtres * ((28-taille_noyau+1)//2)**2
        self.modele_plein = nn.Sequential(
            nn.Linear(self.nb_neurones_du_milieu, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def propagation(self, x, apprentissage=False):
        # Ce code if/else est superflu pour cet exemple, mais sera essentiel 
        # pour un réseau avec «dropout» ou «batchnorm»
        if apprentissage: #  
            self.modele_conv.train()
            self.modele_plein.train()
        else:
            self.modele_conv.eval()
            self.modele_plein.eval()
          
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3
    
    def parametres(self):
        # Cette fonction doit retourner un tuple contenant toutes les variables à optimiser
        return self.modele_conv.parameters(), self.modele_plein.parameters()

Nous pouvons maintenant faire apprendre notre réseau convolutif

In [ ]:
mon_archi = UneArchiPourMNIST(nb_filtres=32, taille_noyau=3)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)

In [ ]:
R.fit(train_x_prime, train_y)

In [ ]:
from sklearn.metrics import accuracy_score
train_pred = R.predict(train_x_prime)
test_pred = R.predict(test_x_prime)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

### Visualisation des filtres appris

In [ ]:
from torchvision.utils import make_grid
def afficher_grille(images):
    plt.figure(figsize=(15,4))
    grid = make_grid(images, pad_value=torch.max(images))
    plt.imshow(grid[0].detach(), cmap=plt.cm.gray)
    plt.colorbar()

In [ ]:
afficher_grille(mon_archi.modele_conv[0].weight)

### Visualisation de la représentation interne du réseau

In [ ]:
def afficher_activations(x, modele, etape):
    image = torch.tensor(x).view(1,1,28,28)
    sous_modele = modele[0:etape]
    print(sous_modele)
    couche = sous_modele(image)
    afficher_grille(couche.transpose(0,1))

In [ ]:
afficher_activations(train_x_prime[0], mon_archi.modele_conv, 1)

In [ ]:
afficher_activations(train_x_prime[0], mon_archi.modele_conv, 2)

In [ ]:
afficher_activations(train_x_prime[0], mon_archi.modele_conv, 3)

************

# À vous de jouer.

Essayez d'autres architectures de réseau de neurones convolutif, et tentez d'interpréter les résultats obtenus. Parmi les différentes possibilités, vous pouvez:
* Changer la taille et le nombre de filtres convolutifs
* Ajouter une ou plusieurs couches de filtres convolutifs dans la première partie du réseau
* Ajouter davantage de couches pleinement connectées dans la seconde partie du réseau
* Ajouter du «Dropout»
* Faire de la «Batchnorm»